In [1]:
cd /data/wlsgur4011/part_assembly/src/pointnext

/data/wlsgur4011/part_assembly/src/pointnext


In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="4"
from jhutil import show_point_clouds
import torch
import numpy as np

import logging
logging.getLogger().setLevel(logging.INFO)

import argparse
import yaml
import os
import sys
import logging
import wandb
from tqdm import tqdm
import torch
import torch.nn as nn
from torch import distributed as dist, multiprocessing as mp
from torch.utils.tensorboard import SummaryWriter
from torch_scatter import scatter
import torch.nn.functional as F
import warnings
import numpy as np
from sklearn.metrics import confusion_matrix
from collections import defaultdict, Counter

torch.backends.cudnn.benchmark = False
warnings.simplefilter(action='ignore', category=FutureWarning)
sys.path.insert(0, "/data/wlsgur4011/part_assembly/src/pointnext/examples")

from openpoints.models import build_model_from_cfg
from openpoints.models.layers import torch_grouping_operation, knn_point
from openpoints.loss import build_criterion_from_cfg
from openpoints.scheduler import build_scheduler_from_cfg
from openpoints.optim import build_optimizer_from_cfg
from openpoints.dataset import build_dataloader_from_cfg, get_class_weights, get_features_by_keys
from openpoints.transforms import build_transforms_from_cfg
from openpoints.utils import AverageMeter, ConfusionMatrix
from openpoints.utils import set_random_seed, save_checkpoint, load_checkpoint, resume_checkpoint, setup_logger_dist, \
    cal_model_parm_nums, Wandb, generate_exp_directory, resume_exp_directory, EasyConfig, dist_utils, find_free_port
from openpoints.models.layers import furthest_point_sample



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
import sys

cfg = EasyConfig()
cfg.load("cfgs/part_assembly/pointnext-s.yaml", recursive=True)

In [4]:
pointnext = build_model_from_cfg(cfg.model).cuda()

[2023-07-10 21:50:05] [INFO] radius: [[0.1], [0.1], [0.25], [0.625], [1.5625]],
 nsample: [[32], [32], [32], [32], [32]]
[2023-07-10 21:50:05] [INFO] NAME: ballquery
normalize_dp: True
radius: 0.1
nsample: 32
[2023-07-10 21:50:05] [INFO] NAME: ballquery
normalize_dp: True
radius: 0.25
nsample: 32
[2023-07-10 21:50:05] [INFO] NAME: ballquery
normalize_dp: True
radius: 0.625
nsample: 32
[2023-07-10 21:50:05] [INFO] NAME: ballquery
normalize_dp: True
radius: 1.5625
nsample: 32


In [7]:
data = {
    "pos": torch.randn((1, 1024, 3)).cuda(),
    "x": torch.randn((1, 1024, 3)).cuda(),
    "y": torch.zeros((1, 1024)).cuda(),
    "heights": torch.zeros((1, 1024, 1)).cuda(),
}


In [8]:
pointnext(data)

RuntimeError: Given groups=1, weight of size [32, 7, 1], expected input[1, 1024, 3] to have 7 channels, but got 1024 channels instead

## PointNet

## PointNet++

## PointNext

## PV-CNN